# Recommendation system by Pandas

## 평점 기준으로 영화 정렬

In [2]:
import pandas as pd

In [9]:
g_cols = ['genre', 'genre_id']
genres =  pd.read_csv('ml-100k/u.genre', sep='|', names=g_cols)
# | 기준으로 나눠줌. name은 사용할 Columns의 리스트가 들어간다.. 

m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', "imdb_url"]
movies = pd.read_csv('ml-100k/u.item', sep='|', names=m_cols, usecols=range(5) ,encoding="latin1")
# usecols는 column들의 subset. 즉 사용할 컬럼만 불러올 때 쓰인다.


u_cols=['user_id','age','sex','occupation','zip_code']
users = pd.read_csv('ml-100k/u.user',sep='|',header= -1,names = u_cols, encoding='latin1') #sep 에 정규식도 들어감.
# header는 숫자에 해당하는 row를 Column명으로 쓴다. 

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols) # 탭 기준 나눔
ratings['date'] = pd.to_datetime(ratings['unix_timestamp'],unit='s') 
#to_datetime 으로 바꿔두면 dt. 할때 aggregation 할때 좋아서 바꿔줌.

In [6]:
movies.head()

,movie_id,title,release_date,video_release_date,imdb_url
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995)


In [10]:
ratings.head()

,user_id,movie_id,rating,unix_timestamp,date
0,196,242,3,881250949,1997-12-04 15:55:49
1,186,302,3,891717742,1998-04-04 19:22:22
2,22,377,1,878887116,1997-11-07 07:18:36
3,244,51,2,880606923,1997-11-27 05:02:03
4,166,346,1,886397596,1998-02-02 05:33:16


In [13]:
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


data = pd.DataFrame.from_csv 이렇게도 csv파일 불러올 수 있다.

In [12]:
movie_rating = pd.merge(movies, ratings) # movies와 ratings 병합(merge)
lens = pd.merge(movie_rating, users) # 위에서 병합한 것에 users를 다시 merge
lens.head()

,movie_id,title,release_date,video_release_date,imdb_url,user_id,rating,unix_timestamp,date,age,sex,occupation,zip_code
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,308,4,887736532,1998-02-17 17:28:52,60,M,retired,95076
1,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,308,5,887737890,1998-02-17 17:51:30,60,M,retired,95076
2,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),308,4,887739608,1998-02-17 18:20:08,60,M,retired,95076
3,7,Twelve Monkeys (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Twelve%20Monk...,308,4,887738847,1998-02-17 18:07:27,60,M,retired,95076
4,8,Babe (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Babe%20(1995),308,5,887736696,1998-02-17 17:31:36,60,M,retired,95076


병합된 데이터를 보면 알겠지만   
pd.merge(movies, ratings)에서는 movie_id를 기준으로 Merge되었고,  
pd.merge(movie_rating, users)에서는 user_id를 기준으로 Merge되었다.

merge, concatenate, join 이 있다.

##### 가장 rating이 많은 영화 찾기

In [16]:
lens.title.value_counts()[:25] #영화별로 평점의 개수를 알려줌. 상위 25개
# lens.title.value_count().nlargest(25) 와 동일함.

# 평점이 많다 -> 인기가 많다.

Star Wars (1977)                             583
Contact (1997)                               509
Fargo (1996)                                 508
Return of the Jedi (1983)                    507
Liar Liar (1997)                             485
English Patient, The (1996)                  481
Scream (1996)                                478
Toy Story (1995)                             452
Air Force One (1997)                         431
Independence Day (ID4) (1996)                429
Raiders of the Lost Ark (1981)               420
Godfather, The (1972)                        413
Pulp Fiction (1994)                          394
Twelve Monkeys (1995)                        392
Silence of the Lambs, The (1991)             390
Jerry Maguire (1996)                         384
Chasing Amy (1997)                           379
Rock, The (1996)                             378
Empire Strikes Back, The (1980)              367
Star Trek: First Contact (1996)              365
Back to the Future (

##### 영화에 대한 평균 평점

In [17]:
lens.groupby('title').rating.mean()
# "title" Column을 기준으로 "rating" Column 데이터에 대한 평균을 내준다.
# 평점 평균

title
'Til There Was You (1997)                                   2.333333
1-900 (1994)                                                2.600000
101 Dalmatians (1996)                                       2.908257
12 Angry Men (1957)                                         4.344000
187 (1997)                                                  3.024390
2 Days in the Valley (1996)                                 3.225806
20,000 Leagues Under the Sea (1954)                         3.500000
2001: A Space Odyssey (1968)                                3.969112
3 Ninjas: High Noon At Mega Mountain (1998)                 1.000000
39 Steps, The (1935)                                        4.050847
8 1/2 (1963)                                                3.815789
8 Heads in a Duffel Bag (1997)                              3.250000
8 Seconds (1994)                                            3.750000
A Chef in Love (1996)                                       4.125000
Above the Rim (1994)        

##### 영화에 대한 평점의 개수

In [24]:
lens.groupby('title').rating.size()
# "title" column을 기준으로 rating row의 개수

title
'Til There Was You (1997)                                     9
1-900 (1994)                                                  5
101 Dalmatians (1996)                                       109
12 Angry Men (1957)                                         125
187 (1997)                                                   41
2 Days in the Valley (1996)                                  93
20,000 Leagues Under the Sea (1954)                          72
2001: A Space Odyssey (1968)                                259
3 Ninjas: High Noon At Mega Mountain (1998)                   5
39 Steps, The (1935)                                         59
8 1/2 (1963)                                                 38
8 Heads in a Duffel Bag (1997)                                4
8 Seconds (1994)                                              4
A Chef in Love (1996)                                         8
Above the Rim (1994)                                          5
Absolute Power (1997)             

In [59]:
lens.groupby('title').rating.count()
# "title" column을 기준으로 rating row 의 개수

title
'Til There Was You (1997)                                     9
1-900 (1994)                                                  5
101 Dalmatians (1996)                                       109
12 Angry Men (1957)                                         125
187 (1997)                                                   41
2 Days in the Valley (1996)                                  93
20,000 Leagues Under the Sea (1954)                          72
2001: A Space Odyssey (1968)                                259
3 Ninjas: High Noon At Mega Mountain (1998)                   5
39 Steps, The (1935)                                         59
8 1/2 (1963)                                                 38
8 Heads in a Duffel Bag (1997)                                4
8 Seconds (1994)                                              4
A Chef in Love (1996)                                         8
Above the Rim (1994)                                          5
Absolute Power (1997)             

In [52]:
lens[lens['title'].isin(['1-900 (1994)'])]
# 위의 데이터에서ㅓ '1-900 (1994)'라는 영화의 rating의 개수가 5개 나왔음을 확인

,movie_id,title,release_date,video_release_date,imdb_url,user_id,rating,unix_timestamp,date,age,sex,occupation,zip_code
1621,1353,1-900 (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?06%20(1994),181,1,878962200,1997-11-08 04:10:00,26,M,executive,21218
67606,1353,1-900 (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?06%20(1994),405,1,885549745,1998-01-23 10:02:25,22,F,healthcare,10019
70234,1353,1-900 (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?06%20(1994),385,4,879440098,1997-11-13 16:54:58,36,M,writer,10003
78225,1353,1-900 (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?06%20(1994),833,3,875035885,1997-09-23 17:31:25,34,M,writer,90019
85038,1353,1-900 (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?06%20(1994),581,4,879641850,1997-11-16 00:57:30,37,M,other,94131


In [54]:
lens.head()

,movie_id,title,release_date,video_release_date,imdb_url,user_id,rating,unix_timestamp,date,age,sex,occupation,zip_code
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,308,4,887736532,1998-02-17 17:28:52,60,M,retired,95076
1,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,308,5,887737890,1998-02-17 17:51:30,60,M,retired,95076
2,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),308,4,887739608,1998-02-17 18:20:08,60,M,retired,95076
3,7,Twelve Monkeys (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Twelve%20Monk...,308,4,887738847,1998-02-17 18:07:27,60,M,retired,95076
4,8,Babe (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Babe%20(1995),308,5,887736696,1998-02-17 17:31:36,60,M,retired,95076


In [3]:
import numpy as np

In [55]:
movie_stats = lens.groupby('title').agg({'rating':[np.size, np.mean]})
# agg : 특정 axis에 대해서 opeation을 써서 Aggregate해줌
# 여기서는 title Column을 기준으로 
# rating에 size 컬럼에 평점의 개수, mean에 평점의 평균점수를 준다.
 

In [58]:
movie_stats.head()

rating          
                            size      mean
title                                     
'Til There Was You (1997)      9  2.333333
1-900 (1994)                   5  2.600000
101 Dalmatians (1996)        109  2.908257
12 Angry Men (1957)          125  4.344000
187 (1997)                    41  3.024390

In [60]:
movie_stats.sort_values(by=[('rating', 'mean')], ascending=False).head(20)
# movie_stats를 rating컬럼의 mean컬럼 기준으로 sorting하며, 내림차순 정렬.

rating          
                                                     size      mean
title                                                              
They Made Me a Criminal (1939)                          1  5.000000
Marlene Dietrich: Shadow and Light (1996)               1  5.000000
Saint of Fort Washington, The (1993)                    2  5.000000
Someone Else's America (1995)                           1  5.000000
Star Kid (1997)                                         3  5.000000
Great Day in Harlem, A (1994)                           1  5.000000
Aiqing wansui (1994)                                    1  5.000000
Santa with Muscles (1996)                               2  5.000000
Prefontaine (1997)                                      3  5.000000
Entertaining Angels: The Dorothy Day Story (1996)       1  5.000000
Pather Panchali (1955)                                  8  4.625000
Some Mother's Son (1996)                                2  4.500000
Maya Lin: A Strong Clear Vision (1994)                  4  4.500000
Anna (1996)                                             2  4.500000
Everest (1998)                                          2  4.500000
Close Shave, A (1995)                                 112  4.491071
Schindler's List (1993)                               298  4.466443
Wrong Trousers, The (1993)                            118  4.466102
Casablanca (1942)                                     243  4.456790
Wallace & Gromit: The Best of Aardman Animation...     67  4.447761

In [63]:
atleast_100 = movie_stats['rating']['size'] >= 100 
# rating개수가 100개 이상인 조건
movie_stats[atleast_100].sort_values(by=[('rating', 'mean')], ascending=False).head()
# 평점 개수가 100개 이상인 조건에서 평점 평균을 내림차순으로 정렬해서 보여줌.

rating          
                                   size      mean
title                                            
Close Shave, A (1995)               112  4.491071
Schindler's List (1993)             298  4.466443
Wrong Trousers, The (1993)          118  4.466102
Casablanca (1942)                   243  4.456790
Shawshank Redemption, The (1994)    283  4.445230

# 추천 시스템 

## 인기순으로 추천하기 (가장 naive한 방법이다)

In [65]:
import pandas as pd
import numpy as np

In [7]:
frame = pd.read_csv('01/rating_final.csv')
cuisine = pd.read_csv('01/chefmozcuisine.csv')

In [67]:
frame.head() # 음식점 평가하는 것. 음식점 추천

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [68]:
cuisine.head()

,placeID,Rcuisine
0,135110,Spanish
1,135109,Italian
2,135107,Latin_American
3,135106,Mexican
4,135105,Fast_Food


In [81]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1161 entries, 0 to 1160
Data columns (total 5 columns):
userID            1161 non-null object
placeID           1161 non-null int64
rating            1161 non-null int64
food_rating       1161 non-null int64
service_rating    1161 non-null int64
dtypes: int64(4), object(1)
memory usage: 45.4+ KB


In [82]:
cuisine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 916 entries, 0 to 915
Data columns (total 2 columns):
placeID     916 non-null int64
Rcuisine    916 non-null object
dtypes: int64(1), object(1)
memory usage: 14.4+ KB


In [84]:
frame.describe()

,placeID,rating,food_rating,service_rating
count,1161.000000,1161.000000,1161.000000,1161.000000
mean,134192.041344,1.199828,1.215332,1.090439
std,1100.916275,0.773282,0.792294,0.790844
min,132560.000000,0.000000,0.000000,0.000000
25%,132856.000000,1.000000,1.000000,0.000000
50%,135030.000000,1.000000,1.000000,1.000000
75%,135059.000000,2.000000,2.000000,2.000000
max,135109.000000,2.000000,2.000000,2.000000


In [83]:
cuisine.describe()

,placeID
count,916.000000
mean,132897.174672
std,923.601711
min,132001.000000
25%,132323.250000
50%,132630.500000
75%,132907.250000
max,135110.000000


In [72]:
# rating을 집계
#most_rated_places = pd.DataFrame(frame.groupby('placeID')['rating'])
# 2개를 merge시킴
merged = pd.merge(frame,cuisine) # placeID Column을 기준으로 Merge해줌.

In [73]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1043 entries, 0 to 1042
Data columns (total 6 columns):
userID            1043 non-null object
placeID           1043 non-null int64
rating            1043 non-null int64
food_rating       1043 non-null int64
service_rating    1043 non-null int64
Rcuisine          1043 non-null object
dtypes: int64(4), object(2)
memory usage: 57.0+ KB


In [74]:
merged.head()

,userID,placeID,rating,food_rating,service_rating,Rcuisine
0,U1077,135085,2,2,2,Fast_Food
1,U1108,135085,1,2,1,Fast_Food
2,U1081,135085,1,2,1,Fast_Food
3,U1056,135085,2,2,2,Fast_Food
4,U1134,135085,2,1,2,Fast_Food


In [76]:
merged.groupby(['Rcuisine', 'placeID']).size().unstack()
# unstack() : pivot이라고도 불리는 unstack은 다중 Index를 지닌 Series객체를 DataFrame으로 바꿔주는 것이다.

# unstack, pivotable

#melt, stack, unstack -> 모양 바꾸는 것.
#pivot은 2개이상 중복될 수 있어서 에러가 뜰 수 있다.
#pivot_table



placeID,132560,132572,132583,132584,132594,132608,132609,132613,132626,132630,...,135073,135074,135075,135079,135085,135086,135088,135104,135106,135109
Rcuisine,,,,,,,,,,,,,,,,,,,,,
American,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Armenian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bakery,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bar_Pub_Brewery,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Breakfast-Brunch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Burgers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN
Cafe-Coffee_Shop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cafeteria,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN


In [80]:
merged.groupby("Rcuisine").agg({'rating':[np.size,np.mean]})
# "Rcuisine" Column기준으로 rating컬럼에 [평점개수, 평점평균]을 준다.
merged.sort_values(by=[('rating')],ascending = False).head(10)
# "rating" Column기준으로 내림차순으로 정렬된 데이터 10개 출력

,userID,placeID,rating,food_rating,service_rating,Rcuisine
0,U1077,135085,2,2,2,Fast_Food
494,U1137,135049,2,2,2,Seafood
539,U1003,132754,2,2,2,Mexican
540,U1061,132754,2,2,2,Mexican
541,U1059,132754,2,2,2,Mexican
542,U1053,135042,2,1,2,Chinese
546,U1076,135042,2,2,2,Chinese
548,U1005,135042,2,1,2,Chinese
549,U1036,135042,2,2,2,Chinese
550,U1071,135042,2,2,2,Chinese


---

## 현재 여기까지 봄..

## 상관관계(correlation)에 따른 추천

In [4]:
geodata = pd.read_csv('01/geoplaces2.csv', encoding='latin1')

In [5]:
geodata.head()

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none
2,135106,22.149709,-100.976093,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rincón de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,?,...,Wine-Beer,only at bar,informal,partially,medium,?,familiar,f,open,none
3,132667,23.752697,-99.163359,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,?,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,t,closed,none
4,132613,23.752903,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,?,...,No_Alcohol_Served,permitted,informal,completely,medium,?,familiar,t,closed,none


In [8]:
frame.food_rating.mean() # 음식 전체의 평점 평균

1.215331610680448

모양바꾸기  
collaborative filtering . user item 으로 구성된 2차원테이블.  
안에는 샀던 횟수가 들어가 있다. 
사용자하고 item 하고 , 그안에 들어갈 세가지 요소만 있으면 추천 알고리즘을 사용할 수 있다.  
판다스에서 추천알고리즘 만드려면 표를 먼저 만들어야한다.   
set index ,unstack , pivot, pivot_table 하는 방법이있다.  
데이터가 채워지지않는 테이블을 sparse 테이블이라 한다.  

In [9]:
places_crosstab = pd.pivot_table(data=frame, values = 'rating', index='userID', columns = 'placeID')
# pivot_table()은 스프레드시트 스타일의 pivot table을 만들어 준다.
# data -> DataFrame객체
# values -> aggregate를 할 column들의 리스트
# index -> pivot table의 index에 대해 groupby를 적용할 키
# column -> pivot table의 column에 대해 groupby를 적용할 키.

In [10]:
places_crosstab.head()

#원래는 melt를 썼는데 recommendation을 할 때는 이렇게 해야한다.

# groupby, stack, unstack, pivot_table, 

#이러한 형태는 sparse -> 데이터가 제대로 채워져있는 것이 많지 않음.

placeID,132560,132561,132564,132572,132583,132584,132594,132608,132609,132613,...,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109
userID,,,,,,,,,,,,,,,,,,,,,
U1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
U1002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN
U1003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U1004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
U1005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# fillna를 통해서 지정하는 값으로 다 채울 수 있다.

places_crosstab.fillna(0) # 빈 데이터 공간에 다 0을 넣어줌.
# NA/NaN 값을 채워주는 함수이다.

#먼저 유사도 구해서 상관계수를 구해야 한다.

placeID,132560,132561,132564,132572,132583,132584,132594,132608,132609,132613,...,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U1002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
U1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U1004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
U1005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U1006,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U1007,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
U1008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
U1009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Item based Collaborative

In [18]:
places_crosstab.head()

placeID,132560,132561,132564,132572,132583,132584,132594,132608,132609,132613,...,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109
userID,,,,,,,,,,,,,,,,,,,,,
U1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
U1002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN
U1003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U1004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
U1005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
places_crosstab.corr().head()
# Columns들 쌍간의 상관관계(correlation) 계산해준다.

placeID,132560,132561,132564,132572,132583,132584,132594,132608,132609,132613,...,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109
placeID,,,,,,,,,,,,,,,,,,,,,
132560,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132561,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132564,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132572,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.090909,NaN,NaN,-0.428571,0.645497,NaN,NaN,NaN,NaN,NaN
132583,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### User based Collaborative

In [17]:
places_crosstab.T.head()

userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
placeID,,,,,,,,,,,,,,,,,,,,,
132560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132561,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132572,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
132583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
xxx = places_crosstab.T.corr()
# T를 통해서 transpose해준다. -> 행과 열을 바꿔준다.

In [14]:
type(xxx)

pandas.core.frame.DataFrame

In [15]:
xxx.head(10)

userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
userID,,,,,,,,,,,,,,,,,,,,,
U1001,1.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-0.301511,NaN,-1.000000,NaN,NaN,NaN,NaN
U1002,1.0,1.000000,-0.522233,NaN,NaN,NaN,NaN,NaN,-0.577350,NaN,...,NaN,NaN,NaN,NaN,NaN,-1.000000,NaN,NaN,NaN,1.0
U1003,NaN,-0.522233,1.000000,NaN,NaN,NaN,NaN,NaN,-0.612372,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.353553,NaN,1.0,NaN,NaN
U1004,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U1005,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U1006,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.534522,NaN,NaN,NaN,NaN
U1007,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U1008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U1009,NaN,-0.577350,-0.612372,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,...,NaN,NaN,NaN,-0.500000,NaN,0.577350,NaN,NaN,0.612372,NaN


유사성(similarity)를 판단하는 것이다. covariance와 correlation.

In [91]:
xxx['U1004'].dropna() # missing data들(NaN)을 제거해준다.

userID
U1004    1.000000
U1024    0.333333
U1032   -1.000000
U1045   -1.000000
U1054   -0.500000
U1071    1.000000
Name: U1004, dtype: float64

1004는 1071과 제일 가깝다. 1071의 구매내역을 똑같이 추천해준다.
이런게 Collaborative Filtering. 우리는 가져다 쓰기만했을 뿐.

In [ ]:
#corr()과 corrwith() 의 차이점. corr()는 칼럼들 corrwith()는두개의 데이터프레임 

similar_to_Tortas = places_crosstab.corrwith(Tortas_rating)

corr_Tortas = pd.DataFrame(similar_to_Tortas, columns= ['PearsonR'])
corr_Tortas.dropna(inplace=True)
corr_Tortas.head()

#Correlation을 이용한다.
# 이렇게 Collaborative 구현할 수 있다.
# 이런 방식으로 한다는 것 -> 지금 실행하면 에러가 날 것이다.

---

In [120]:
!python -V

Python 3.6.3 :: Anaconda custom (64-bit)
